In [22]:
import re
import random

data_path = "model/human_text.txt"
data_path2 = "model/robot_text.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')
lines = [re.sub(r"\[\w+\]",'hola',line) for line in lines]
lines = [" ".join(re.findall(r"\w+",line.lower())) for line in lines]
lines2 = [re.sub(r"\[\w+\]",'',line) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line.lower())) for line in lines2]
# Grouping lines by response pair
pairs = list(zip(lines,lines2))
#random.shuffle(pairs)

In [23]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [24]:
print(pairs[:5])
print(input_docs[:5])

[('hola buenos días', 'buenos días en que puedo servirle'), ('tienes domicilio', 'claro que si que deseas pedir a domicilio'), ('que cuesta la arepa de huevo', 'la arepa tiene un costo de 2000 pesos sin incluir el domicilio'), ('y vendes chocolatico caliente', 'claro el chocolate un costo de 1000 sin leche'), ('y con leche', 'con lehe tiene un costo de 2000 pesos')]
['hola buenos días', 'tienes domicilio', 'que cuesta la arepa de huevo', 'y vendes chocolatico caliente', 'y con leche']


In [25]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 600
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('model/training_model.h5')

Epoch 1/600
21/21 [==============================] - 6s 104ms/step - loss: 1.3921 - accuracy: 0.0269 - val_loss: 1.2717 - val_accuracy: 0.0286
Epoch 2/600
21/21 [==============================] - 1s 55ms/step - loss: 1.2707 - accuracy: 0.0332 - val_loss: 1.2691 - val_accuracy: 0.0306
Epoch 3/600
21/21 [==============================] - 1s 57ms/step - loss: 1.2417 - accuracy: 0.0356 - val_loss: 1.2743 - val_accuracy: 0.0326
Epoch 4/600
21/21 [==============================] - 1s 56ms/step - loss: 1.2723 - accuracy: 0.0340 - val_loss: 1.2727 - val_accuracy: 0.0347
Epoch 5/600
21/21 [==============================] - 1s 56ms/step - loss: 1.2233 - accuracy: 0.0388 - val_loss: 1.2727 - val_accuracy: 0.0372
Epoch 6/600
21/21 [==============================] - 1s 55ms/step - loss: 1.1631 - accuracy: 0.0396 - val_loss: 1.2806 - val_accuracy: 0.0342
Epoch 7/600
21/21 [==============================] - 1s 56ms/step - loss: 1.2336 - accuracy: 0.0411 - val_loss: 1.2770 - val_accuracy: 0.0377
Epoch

21/21 [==============================] - 1s 55ms/step - loss: 0.8654 - accuracy: 0.1017 - val_loss: 1.2719 - val_accuracy: 0.0658
Epoch 59/600
21/21 [==============================] - 1s 57ms/step - loss: 0.8336 - accuracy: 0.1036 - val_loss: 1.2656 - val_accuracy: 0.0648
Epoch 60/600
21/21 [==============================] - 1s 55ms/step - loss: 0.8514 - accuracy: 0.0949 - val_loss: 1.2775 - val_accuracy: 0.0663
Epoch 61/600
21/21 [==============================] - 1s 55ms/step - loss: 0.9069 - accuracy: 0.1021 - val_loss: 1.2671 - val_accuracy: 0.0699
Epoch 62/600
21/21 [==============================] - 1s 55ms/step - loss: 0.8755 - accuracy: 0.1081 - val_loss: 1.2701 - val_accuracy: 0.0688
Epoch 63/600
21/21 [==============================] - 1s 55ms/step - loss: 0.8103 - accuracy: 0.0994 - val_loss: 1.2649 - val_accuracy: 0.0709
Epoch 64/600
21/21 [==============================] - 1s 55ms/step - loss: 0.8041 - accuracy: 0.1076 - val_loss: 1.2550 - val_accuracy: 0.0734
Epoch 65/600

21/21 [==============================] - 1s 56ms/step - loss: 0.5885 - accuracy: 0.1693 - val_loss: 1.2244 - val_accuracy: 0.0821
Epoch 116/600
21/21 [==============================] - 1s 55ms/step - loss: 0.6164 - accuracy: 0.1623 - val_loss: 1.2088 - val_accuracy: 0.0867
Epoch 117/600
21/21 [==============================] - 1s 56ms/step - loss: 0.6150 - accuracy: 0.1602 - val_loss: 1.2201 - val_accuracy: 0.0872
Epoch 118/600
21/21 [==============================] - 1s 56ms/step - loss: 0.5251 - accuracy: 0.1653 - val_loss: 1.2283 - val_accuracy: 0.0816
Epoch 119/600
21/21 [==============================] - 1s 55ms/step - loss: 0.5998 - accuracy: 0.1684 - val_loss: 1.2151 - val_accuracy: 0.0852
Epoch 120/600
21/21 [==============================] - 1s 55ms/step - loss: 0.5230 - accuracy: 0.1637 - val_loss: 1.2117 - val_accuracy: 0.0841
Epoch 121/600
21/21 [==============================] - 1s 55ms/step - loss: 0.5818 - accuracy: 0.1690 - val_loss: 1.2137 - val_accuracy: 0.0867
Epoch 

Epoch 172/600
21/21 [==============================] - 1s 56ms/step - loss: 0.3976 - accuracy: 0.2034 - val_loss: 1.2275 - val_accuracy: 0.0806
Epoch 173/600
21/21 [==============================] - 1s 56ms/step - loss: 0.4048 - accuracy: 0.1928 - val_loss: 1.2274 - val_accuracy: 0.0801
Epoch 174/600
21/21 [==============================] - 1s 55ms/step - loss: 0.4037 - accuracy: 0.1921 - val_loss: 1.2354 - val_accuracy: 0.0831
Epoch 175/600
21/21 [==============================] - 1s 55ms/step - loss: 0.4365 - accuracy: 0.2014 - val_loss: 1.2281 - val_accuracy: 0.0826
Epoch 176/600
21/21 [==============================] - 1s 55ms/step - loss: 0.3610 - accuracy: 0.1941 - val_loss: 1.2229 - val_accuracy: 0.0841
Epoch 177/600
21/21 [==============================] - 1s 56ms/step - loss: 0.4415 - accuracy: 0.2000 - val_loss: 1.2245 - val_accuracy: 0.0847
Epoch 178/600
21/21 [==============================] - 1s 55ms/step - loss: 0.3928 - accuracy: 0.1973 - val_loss: 1.2462 - val_accuracy:

Epoch 229/600
21/21 [==============================] - 1s 57ms/step - loss: 0.3428 - accuracy: 0.2147 - val_loss: 1.2618 - val_accuracy: 0.0801
Epoch 230/600
21/21 [==============================] - 1s 55ms/step - loss: 0.3442 - accuracy: 0.2109 - val_loss: 1.2512 - val_accuracy: 0.0836
Epoch 231/600
21/21 [==============================] - 1s 55ms/step - loss: 0.3555 - accuracy: 0.2118 - val_loss: 1.2544 - val_accuracy: 0.0821
Epoch 232/600
21/21 [==============================] - 1s 55ms/step - loss: 0.3102 - accuracy: 0.2193 - val_loss: 1.2594 - val_accuracy: 0.0806
Epoch 233/600
21/21 [==============================] - 1s 54ms/step - loss: 0.3436 - accuracy: 0.2212 - val_loss: 1.2573 - val_accuracy: 0.0811
Epoch 234/600
21/21 [==============================] - 1s 55ms/step - loss: 0.2892 - accuracy: 0.2071 - val_loss: 1.2706 - val_accuracy: 0.0811
Epoch 235/600
21/21 [==============================] - 1s 55ms/step - loss: 0.3313 - accuracy: 0.2167 - val_loss: 1.2325 - val_accuracy:

...

Epoch 343/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2738 - accuracy: 0.2353 - val_loss: 1.3039 - val_accuracy: 0.0806
Epoch 344/600
21/21 [==============================] - 1s 57ms/step - loss: 0.2584 - accuracy: 0.2328 - val_loss: 1.2997 - val_accuracy: 0.0790
Epoch 345/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2466 - accuracy: 0.2306 - val_loss: 1.2984 - val_accuracy: 0.0780
Epoch 346/600
21/21 [==============================] - 1s 57ms/step - loss: 0.2364 - accuracy: 0.2196 - val_loss: 1.3073 - val_accuracy: 0.0770
Epoch 347/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2544 - accuracy: 0.2273 - val_loss: 1.2757 - val_accuracy: 0.0811
Epoch 348/600
21/21 [==============================] - 1s 57ms/step - loss: 0.2540 - accuracy: 0.2333 - val_loss: 1.2943 - val_accuracy: 0.0785
Epoch 349/600
21/21 [==============================] - 1s 58ms/step - loss: 0.2757 - accuracy: 0.2323 - val_loss: 1.3159 - val_accuracy:

Epoch 400/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2676 - accuracy: 0.2355 - val_loss: 1.3044 - val_accuracy: 0.0801
Epoch 401/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2076 - accuracy: 0.2333 - val_loss: 1.3286 - val_accuracy: 0.0729
Epoch 402/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2425 - accuracy: 0.2308 - val_loss: 1.3119 - val_accuracy: 0.0770
Epoch 403/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2237 - accuracy: 0.2272 - val_loss: 1.3201 - val_accuracy: 0.0770
Epoch 404/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2241 - accuracy: 0.2389 - val_loss: 1.3140 - val_accuracy: 0.0785
Epoch 405/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2486 - accuracy: 0.2242 - val_loss: 1.3247 - val_accuracy: 0.0801
Epoch 406/600
21/21 [==============================] - 1s 55ms/step - loss: 0.2438 - accuracy: 0.2358 - val_loss: 1.3273 - val_accuracy:

Epoch 457/600
21/21 [==============================] - 1s 61ms/step - loss: 0.2048 - accuracy: 0.2298 - val_loss: 1.3489 - val_accuracy: 0.0755
Epoch 458/600
21/21 [==============================] - 1s 58ms/step - loss: 0.2210 - accuracy: 0.2522 - val_loss: 1.3605 - val_accuracy: 0.0755
Epoch 459/600
21/21 [==============================] - 1s 55ms/step - loss: 0.2160 - accuracy: 0.2370 - val_loss: 1.3613 - val_accuracy: 0.0770
Epoch 460/600
21/21 [==============================] - 1s 60ms/step - loss: 0.2028 - accuracy: 0.2302 - val_loss: 1.3345 - val_accuracy: 0.0785
Epoch 461/600
21/21 [==============================] - 1s 58ms/step - loss: 0.2186 - accuracy: 0.2434 - val_loss: 1.3633 - val_accuracy: 0.0755
Epoch 462/600
21/21 [==============================] - 1s 57ms/step - loss: 0.2183 - accuracy: 0.2317 - val_loss: 1.3702 - val_accuracy: 0.0765
Epoch 463/600
21/21 [==============================] - 1s 58ms/step - loss: 0.2059 - accuracy: 0.2295 - val_loss: 1.3401 - val_accuracy:

Epoch 514/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1867 - accuracy: 0.2304 - val_loss: 1.3814 - val_accuracy: 0.0765
Epoch 515/600
21/21 [==============================] - 1s 57ms/step - loss: 0.1862 - accuracy: 0.2343 - val_loss: 1.3615 - val_accuracy: 0.0765
Epoch 516/600
21/21 [==============================] - 1s 56ms/step - loss: 0.2155 - accuracy: 0.2590 - val_loss: 1.3684 - val_accuracy: 0.0780
Epoch 517/600
21/21 [==============================] - 1s 57ms/step - loss: 0.1927 - accuracy: 0.2376 - val_loss: 1.3748 - val_accuracy: 0.0750
Epoch 518/600
21/21 [==============================] - 1s 57ms/step - loss: 0.1830 - accuracy: 0.2347 - val_loss: 1.3799 - val_accuracy: 0.0750
Epoch 519/600
21/21 [==============================] - 1s 57ms/step - loss: 0.2092 - accuracy: 0.2419 - val_loss: 1.3860 - val_accuracy: 0.0719
Epoch 520/600
21/21 [==============================] - 1s 70ms/step - loss: 0.1835 - accuracy: 0.2309 - val_loss: 1.3734 - val_accuracy:

Epoch 571/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1941 - accuracy: 0.2544 - val_loss: 1.3703 - val_accuracy: 0.0750
Epoch 572/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1694 - accuracy: 0.2454 - val_loss: 1.3809 - val_accuracy: 0.0745
Epoch 573/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1457 - accuracy: 0.2310 - val_loss: 1.3588 - val_accuracy: 0.0724
Epoch 574/600
21/21 [==============================] - 1s 56ms/step - loss: 0.1836 - accuracy: 0.2568 - val_loss: 1.3830 - val_accuracy: 0.0739
Epoch 575/600
21/21 [==============================] - 1s 55ms/step - loss: 0.1669 - accuracy: 0.2365 - val_loss: 1.4080 - val_accuracy: 0.0755
Epoch 576/600
21/21 [==============================] - 1s 57ms/step - loss: 0.1655 - accuracy: 0.2422 - val_loss: 1.3930 - val_accuracy: 0.0719
Epoch 577/600
21/21 [==============================] - 1s 55ms/step - loss: 0.1680 - accuracy: 0.2379 - val_loss: 1.4114 - val_accuracy:

In [26]:
training_model.save('model/training_model2.h5')

In [27]:
from keras.models import load_model
training_model = load_model('model/training_model2.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
    while not stop_condition:
          #Predicting output tokens with probabilities and states
          output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    #Choosing the one with highest probability
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_token = reverse_target_features_dict[sampled_token_index]
          decoded_sentence += " " + sampled_token
    #Stop if hit max length or found the stop token
          if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
    #Update the target sequence
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          #Update states
          states_value = [hidden_state, cell_state]
    return decoded_sentence

In [28]:
class ChatBot:
  negative_responses = ("no", "negativo", "nada", "nop", "nopi")
  exit_commands = ("chao", "adios", "nos vemos",
                 "suerte", "chaito", "bye", "chaolin")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hola, bienvenido en que te puedo ayudar?\n")
    
    if user_response in self.negative_responses:
      print("Ok, perfecto, que tengas un hermoso día!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Que tengas un hermoso día!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

Hola, bienvenido en que te puedo ayudar?
Hola, buenos días
 también tenemos platos a la 
hola, buenos dias
 buenos días en que te puedo servirle 
chao
Que tengas un hermoso día!


In [ ]:
chatbot.start_chat()